In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.09360599517822266
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 16.98013167290981
200000 29.03495896235018
300000 29.41877502649201
400000 28.654616866032622
500000 28.538562018952753
600000 29.494893573997743
700000 28.974018229615226
800000 28.785684235342597
900000 30.08247547108622
1000000 28.32129517565925
1100000 28.344305695967293
1200000 12.24413692861968
1300000 24.341719566565203
1400000 28.02000025984318
1500000 28.36229693350512
1600000 28.076312164456834
1700000 28.324278466737283
1800000 29.921043368457205
1900000 28.608832872295206
2000000 28.336603593270933
2100000 28.177744681271854
2200000 28.880250178666795
2300000 28.804711057281644
2400000 7.475138806769345
2500000 30.27748171028171
2600000 35.816422134793804
2700000 34.987223609948096
2800000 35.05471568358101
2900000 35.303433799757414
3000000 34

29500000 16.137731842093377
29600000 26.38397232366855
29700000 27.471451654531794
29800000 27.71164980783317
29900000 28.58277175378894
30000000 27.14808056216127
30100000 28.931027762187213
30200000 28.625930462207137
30300000 28.510400889472194
30400000 12.068899572262934
30500000 28.543370997822116
30600000 28.814900081748306
30700000 30.243704067576775
30800000 25.270213609768607
30900000 14.725776018318323
31000000 28.785700133622946
31100000 29.32324604361906
31200000 28.52924862209919
31300000 27.99482135942865
31400000 27.78502770620172
31500000 27.423317691065105
31600000 26.92432743660097
31700000 25.997763074249452
31800000 7.764003332394115
31900000 27.151678956904547
32000000 25.659438301437422
32100000 27.030647642506715
32200000 26.980830565388775
32300000 28.038568286253145
32400000 27.64180963755464
32500000 28.775098680945426
32600000 27.97567733124653
32700000 19.16973352085689
32800000 21.249330478031194
32900000 28.402701447006166
33000000 29.38356813236979
331000

59300000 11.711304635972821
59400000 24.785753273056567
59500000 26.037292034467825
59600000 25.86410684170997
59700000 23.933337846642054
59800000 23.84635190926613
59900000 23.859472056542817
60000000 13.869268744796088
60100000 17.56674193584461
60200000 24.263128144620076
60300000 23.698608761572537
60400000 23.9924087555399
60500000 25.072669993048184
60600000 24.741040327379196
60700000 21.52504390415897
60800000 16.773197478849298
60900000 27.558699328713455
61000000 26.818305132868662
61100000 26.722996630415143
61200000 27.184031705952655
61300000 27.78040348809759
61400000 26.87002857091437
61500000 27.58612600013621
61600000 25.904489549987208
61700000 10.236439322653505
61800000 26.195933363475497
61900000 25.926343143270834
62000000 25.058367740763188
62100000 24.51080770289992
62200000 24.933285467003127
62300000 24.68045305032547
62400000 6.194504613067149
62500000 25.363648642550938
62600000 23.664429067650005
62700000 24.326579261699678
62800000 24.14698025143163
62900

89100000 22.797492760833265
89200000 23.52047875870245
89300000 22.562519019280373
89400000 23.721100274964922
89500000 24.148279007001378
89600000 23.976794631362747
89700000 24.353806095926515
89800000 23.660382061313634
89900000 23.456893806715236
90000000 22.318607650754725
90100000 12.16962612672261
90200000 18.16357289822145
90300000 21.281581064830586
90400000 22.398619902941906
90500000 21.676177070228363
90600000 9.43383078031979
90700000 19.54765022537356
90800000 21.88964519846722
90900000 21.304013621247705
91000000 21.298662410056973
91100000 10.632681250052485
91200000 20.374512833232476
91300000 22.950991432596158
91400000 22.23150372781382
91500000 23.536760015030247
91600000 23.139229163647624
91700000 23.85650496227299
91800000 23.972805151515022
91900000 23.47699938265019
92000000 23.353732796595743
92100000 24.05943686908702
92200000 23.223692294324973
92300000 22.41687033318573
92400000 22.095158895361863
92500000 7.656155815799965
92600000 21.452083916831672
92700

118100000 19.98499420105139
118200000 20.640566862991065
118300000 19.46131907564972
118400000 19.972898027184527
118500000 17.016334124461252
118600000 9.534975019642264
118700000 20.50056013809673
118800000 13.442409253993608
118900000 11.294657261000012
119000000 19.782389582124775
119100000 12.16519452769797
119200000 15.422074821982374
119300000 20.080461823588156
119400000 21.563048560823272
119500000 19.87268841296111
119600000 19.83888936530546
119700000 20.90536575513938
119800000 20.895467228656894
119900000 21.621441483457367
120000000 21.196817035312495
120100000 21.967521512869155
120200000 21.294684640384798
120300000 20.777651318524487
120400000 19.821983756556612
120500000 20.24827214535651
120600000 20.259732907363052
120700000 18.76130789118715
120800000 20.194171454604344
120900000 12.175440517462114
121000000 13.539394631586378
121100000 19.190188898369573
121200000 5.4732579765095215
121300000 18.721772186948993
121400000 17.689580642430904
121500000 10.31725801480

...

261400000 9.63972792662979
261500000 8.156132723804403
261600000 5.513307190800531
261700000 9.62812400284174
261800000 9.209095183712085
261900000 9.073880193247732
262000000 9.173432208336795
262100000 8.783415402056114
262200000 8.722999004803036
262300000 7.020682547495367
262400000 4.559375883965071
262500000 8.515324827395116
262600000 8.747634965341893
262700000 9.090015598393824
262800000 9.045248902648948
262900000 9.319423348117661
263000000 9.650523932721851
263100000 7.759473170679243
263200000 5.468815200061362
263300000 9.71759038996685
263400000 9.89128436117604
263500000 9.732106048478832
263600000 9.748683654640816
263700000 9.53801431438031
263800000 9.754316379923266
263900000 3.7103914878409077
264000000 10.308006398111116
264100000 9.579228077141009
264200000 9.279700386867255
264300000 9.245474167354088
264400000 9.152984845371899
264500000 8.724557604055493
264600000 8.746722463034876
264700000 3.4921747526839315
264800000 8.117238064238936
264900000 8.9770333530

290800000 7.58834597313655
290900000 7.641808287754239
291000000 7.670821016940161
291100000 7.466240893110879
291200000 7.742128049694103
291300000 7.7210701502144685
291400000 7.812240786090879
291500000 7.924247396303876
291600000 2.84254338327378
291700000 8.056263757845445
291800000 7.625232927841861
291900000 4.204094288597406
292000000 6.876836542387936
292100000 7.718310164353603
292200000 7.512562765968462
292300000 7.7232282438386415
292400000 7.513640907884516
292500000 7.426850244543053
292600000 8.637928327089679
292700000 9.621384665128584
292800000 7.688617276828087
292900000 4.823902627700887
293000000 4.617860818871015
293100000 7.530757434885832
293200000 7.676070197859316
293300000 7.645436480724148
293400000 7.612159875698655
293500000 7.9349887917049475
293600000 7.77391799936038
293700000 7.60996843265144
293800000 7.717804575074328
293900000 7.251977374350131
294000000 3.541342713896836
294100000 7.576443272302815
294200000 6.059209305938363
294300000 4.965897786

319700000 2.284712527958922
319800000 2.437606995630344
319900000 2.5032612734763937
320000000 2.7544284968483423
320100000 2.45375247292754
320200000 2.544189780671479
320300000 2.2469694638694975
320400000 2.269833996162671
320500000 2.4937071644393685
320600000 2.31529858478727
320700000 1.977254654628882
320800000 2.354933500373105
320900000 2.070104941402468
321000000 1.7536564987503411
321100000 1.2645633767880975
321200000 1.2503198031048801
321300000 1.2497259097989413
321400000 1.476462805316049
321500000 2.391067242796275
321600000 2.6958460223264056
321700000 2.142644611987521
321800000 1.930591169640995
321900000 2.2797294026570434
322000000 2.343387010023572
322100000 2.3234731922481595
322200000 2.2729521055445203
322300000 2.594529202609831
322400000 2.4423637457507623
322500000 2.4949370811809
322600000 2.395280453251044
322700000 2.2291292373542984
322800000 2.1726378489023266
322900000 2.0468869596324404
323000000 2.175001336960797
323100000 2.4416880355792276
3232000

348300000 1.6033545569754881
348400000 1.3760226473850312
348500000 1.325381435373044
348600000 1.2999941394952694
348700000 1.3768427740780076
348800000 1.1278362570500533
348900000 1.2063902775099695
349000000 1.388853959676548
349100000 1.403278469726475
349200000 0.9969331936661164
349300000 0.6744161163504482
349400000 0.7065378532040469
349500000 0.7242060441002091
349600000 0.8886905883841079
349700000 1.0729616861992968
349800000 1.4180431662985906
349900000 1.1678894155335962
350000000 1.275335011505952
350100000 1.3006594790370583
350200000 1.2129555740444302
350300000 1.2199044267091426
350400000 1.3820404763662926
350500000 1.3232290325995941
350600000 1.4097271236341018
350700000 1.2948667546271333
350800000 1.3433773394730626
350900000 1.2235468449654094
351000000 1.2087489103480147
351100000 1.2078820767112612
351200000 1.2171137224287432
351300000 1.2043759598394772
351400000 1.2320639146426082
351500000 0.9892037231062976
351600000 0.8356119272628261
351700000 0.726726

376100000 0.055732854930166394
376200000 0.05059851289488805
376300000 0.057588293553024535
376400000 0.05625262584526539
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.24840094167570206
376600000 0.31777909655746023
376700000 0.32468699621769986
376800000 0.3130207760708041
376900000 0.2737881857568026
377000000 0.28502018645511334
377100000 0.2884804278696961
377200000 0.23660566496937274
377300000 0.23348301753668985
377400000 0.24040160452441575
377500000 0.24354531334008972
377600000 0.23435466617623765
377700000 0.20870953546909027
377800000 0.22514937820384304
377900000 0.22919289687423838
378000000 0.20526225933194425
378100000 0.21668258136193155
378200000 0.22765195621426434
378300000 0.22837835291169287
378400000 0.2151770204558492
378500000 0.23298123941000037
378600000 0.23016664581282337
378700000 0.2524601629960636
378800000 0.1868709518700037
378900000 0.04245085027916233
379000000 0.036433090888202194
379100000 0.04663167277656264
379